In [1]:
#учит LLM в сборе. То есть приделываем в конец feature head и так и учим.

In [2]:
install = False
if install:
    !conda install -c anaconda git -y

    !pip install deepspeed==0.14.4

    !pip install --upgrade pip
    !pip install --upgrade transformers==4.46.2 tyro==0.9.8 triton==2.3.1 trl==0.12.0

    !pip install -v "xformers==0.0.29.post1"
    !pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
    !pip install --no-deps -v "peft==0.13.2"
    !pip install --no-deps packaging ninja einops flash-attn bitsandbytes==0.44.1
    !pip install accelerate==0.34.2
    #!curl -fsSL https://ollama.com/install.sh | sh
    !pip uninstall peft -y
    !pip install -v "peft==0.13.2"
    

In [3]:
!pip freeze

absl-py==2.2.2
accelerate==0.34.2
aiohappyeyeballs==2.6.1
aiohttp==3.11.14
aiosignal==1.3.2
annotated-types==0.7.0
anyio==4.9.0
argon2-cffi==23.1.0
argon2-cffi-bindings==21.2.0
arrow==1.3.0
asttokens==3.0.0
async-lru==2.0.5
attrs==25.3.0
babel==2.17.0
beautifulsoup4==4.13.3
bitsandbytes==0.44.1
bleach==6.2.0
certifi==2025.1.31
cffi==1.17.1
charset-normalizer==3.4.1
click==8.1.8
comm==0.2.2
cut-cross-entropy==25.1.1
dataclasses-json==0.6.7
datasets==3.4.1
debugpy==1.8.13
decorator==5.2.1
deepspeed==0.14.4
defusedxml==0.7.1
dill==0.3.8
docstring_parser==0.16
einops==0.8.1
executing==2.2.0
fastjsonschema==2.21.1
filelock==3.18.0
flash_attn==2.7.4.post1
fqdn==1.5.1
frozenlist==1.5.0
fsspec==2024.12.0
gigachat==0.1.39.post2
GPUtil==1.4.0
greenlet==3.2.3
h11==0.14.0
hf_transfer==0.1.9
hjson==3.1.0
httpcore==1.0.7
httpx==0.28.1
httpx-sse==0.4.1
huggingface-hub==0.29.3
idna==3.10
ipykernel==6.29.5
ipython==9.0.2
ipython_pygments_lexers==1.1.1
ipywidgets==8.1.5
isoduration==20.11.0
jedi==0.19.2

In [4]:
import sys
import time
import json
import os
from collections import OrderedDict
from pathlib import Path
from collections import deque
import os, pickle, random
import psutil
import warnings
import codecs
import numpy as np
import pandas as pd
import requests
from typing import Dict, List, Optional
import torch.nn as nn
import torch.nn.functional as F

import torch
import torch.nn.functional as F

from torch.utils.data import Dataset, DataLoader

import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from transformers import Trainer, TrainingArguments
from transformers import AutoTokenizer, AutoModel
from transformers import StoppingCriteria, StoppingCriteriaList
from torch import cuda, LongTensor, FloatTensor
from peft import PeftModel, PeftConfig, PeftModelForCausalLM

import ensembles
from data_tools import InstructDatasetR

device = torch.device("cuda:0")

In [5]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # Теперь для PyTorch доступен только GPU 0

In [6]:
# Проверяем доступность CUDA и количество GPU
#если надо, можем переключиться на другую GPU
if 0:
    if torch.cuda.is_available():
        if torch.cuda.device_count() > 1:  # Если есть более 1 GPU
            device = torch.device("cuda:1")  # Явно указываем устройство 1
            print(f"Переключились на GPU 1")
        else:
            print("Доступен только один GPU (номер 0), переключение невозможно")
    else:
        print("CUDA недоступно")

In [7]:
torch.tensor(1).to(device)

tensor(1, device='cuda:0')

In [8]:
import time
#time.sleep(7*60)

In [9]:
start_train = False#Запускаем ли мы сейчас обучение с нуля, или с какого-то чекпоинта


bits_per_number = 4#Насколько сильно квантуем модель
use_gguf = False#Можно использовать gguf
gguf_folder_name = "gguf_31"#из этой папки


cardinality = 128256#размер словаря токенов
learnable_linear_model = False#учим ли мы линейную субмодель
learnable_all = False#учим ли мы вообще весь трансформер (это очень тяжело по GPU-памяти, я не осилил)

cfg_switch = 2
if cfg_switch == 1:
    #ЛЕГКОВЕСНЫЙ ТРАНСФОРМЕР
    model_name = "unsloth/Llama-3.2-1B-Instruct-bnb-4bit"
    architecture = 'B'
    opt_type = 'adam'
    mode = 'pretrain'#'finetune', 'pretrain'
    #1.403
else:
    #ТЯЖЁЛЫЙ ТРАНСФОРМЕР
    model_name = "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit"
    architecture = 'N'#'L'
    opt_type = 'adam'#'adam'
    mode = 'finetune'


tokenizer_name = model_name
padding_token = 128009
forbidden_tokens_list = [padding_token]
seed = int(np.random.rand() * 1000000)#random seed for data sampling

In [10]:
use_memnets = True
use_moe = False
memnet_params = {}
if model_name == "unsloth/Llama-3.2-1B-Instruct-bnb-4bit":
    if architecture == 'A':
        embedding_size = 2048#это просто свойство исходной сетки
        #сколько субмоделей выбрасывается из forward на обучении. Чем больше, тем лучше защита от оверфита. Если одна-две субмодели, net_dropout лучше занулять
        net_dropout_rate = 0.0#если десятки субмоделей и хотим огромную защиту от оверфита, то это число надо проставлять в 0.9-0.95
        #дропаут внутри субмоделей
        individ_dropout_rate = 0.05
        conservativity = 0.05#насколько сильно подавляем отклонение от старой стратегии
        accum_batch = 1000#130#110#60#для дачала сделайте 2
        lr = 1e-6#1e-3
        
        layer_configs = [2048, 2048, 2048, 2048, 2048, 2048, 2048, 2048, 2048, 2048, 2048, 1024, 1024, 1024, 1024, 1024, 512, 512, 512, 512, 512, 512, 512]#версия A
        composition_size = 1#число субмоделей класса ResNet. Чем больше их, тем больше или защита от оверфита (если большой net_dropout), или ёмкость
        sample_features = 1.#какая доля фичей приходит в субмодель. По дефолту ставится 0.4-0.6, но это если субмоделей как минимум 15. Сейчас их 1.
        path2model = "ern_model_A_composed.pth"
        max_tokens_in_loss = 1300#сколько пар эмбеддинг-токен проходит через tail adapter за один forward
        batch_size = 3#2
    elif architecture == 'B':
        embedding_size = 2048#это просто свойство исходной сетки
        #сколько субмоделей выбрасывается из forward на обучении. Чем больше, тем лучше защита от оверфита. Если одна-две субмодели, net_dropout лучше занулять
        net_dropout_rate = 0.5#если десятки субмоделей и хотим огромную защиту от оверфита, то это число надо проставлять в 0.9-0.95
        #дропаут внутри субмоделей
        individ_dropout_rate = 0.05
        conservativity = 0.05#насколько сильно подавляем отклонение от старой стратегии
        accum_batch = 100#130#110#60#для дачала сделайте 2
        lr = 1e-6#1e-3
        
        layer_configs = [2048, 2048, 2048, 2048, 2048, 2048, 2048, 2048, 2048, 2048, 2048, 1024, 1024, 1024, 1024, 1024, 512, 512, 512, 512, 512, 512, 512]#версия A
        composition_size = 16#число субмоделей класса ResNet. Чем больше их, тем больше или защита от оверфита (если большой net_dropout), или ёмкость
        sample_features = 1.#какая доля фичей приходит в субмодель. По дефолту ставится 0.4-0.6, но это если субмоделей как минимум 15. Сейчас их 1.
        path2model = "ern_model_B_composed.pth"
        max_tokens_in_loss = 1300#сколько пар эмбеддинг-токен проходит через tail adapter за один forward
        batch_size = 2#2
        
elif model_name == "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit":
    embedding_size = 4096#это просто свойство исходной сетки

    if architecture == 'D':
        #сколько субмоделей выбрасывается из forward на обучении. Чем больше, тем лучше защита от оверфита. Если одна-две субмодели, net_dropout лучше занулять
        net_dropout_rate = 0.0#если десятки субмоделей и хотим огромную защиту от оверфита, то это число надо проставлять в 0.9-0.95
        #дропаут внутри субмоделей
        individ_dropout_rate = 0.2
        conservativity = 0.12#насколько сильно подавляем отклонение от старой стратегии
        accum_batch = 4200#60#для дачала сделайте 2
        lr = 3e-6
        
        layer_configs = [1024, 512, 512, 512, 256, 256, 256, 256, 128, 128, 128, 128]#версия C
        layer_configs = [2048, 1024, 1024, 512, 512, 512, 256, 256, 256, 256, 256, 256, 256, 256, 256]#версия D
        composition_size = 1#число субмоделей класса ResNet. Чем больше их, тем больше или защита от оверфита (если большой net_dropout), или ёмкость
        sample_features = 1.#какая доля фичей приходит в субмодель. По дефолту ставится 0.4-0.6, но это если субмоделей как минимум 15. Сейчас их 1.
        path2model = "ern_model_D_composed.pth"
        max_tokens_in_loss = 5500#сколько пар эмбеддинг-токен проходит через tail adapter за один forward
    elif architecture == 'E':
        #сколько субмоделей выбрасывается из forward на обучении. Чем больше, тем лучше защита от оверфита. Если одна-две субмодели, net_dropout лучше занулять
        net_dropout_rate = 0.0#если десятки субмоделей и хотим огромную защиту от оверфита, то это число надо проставлять в 0.9-0.95
        #дропаут внутри субмоделей
        individ_dropout_rate = 0.05
        conservativity = 0.15#насколько сильно подавляем отклонение от старой стратегии
        accum_batch = 2300#60#для дачала сделайте 2
        lr = 1e-3
        
        layer_configs = [2048, 2048, 1024, 1024, 1024, 512, 512, 512, 512, 512, 512, 512, 256, 256, 256, 256]#версия E
        composition_size = 1#число субмоделей класса ResNet. Чем больше их, тем больше или защита от оверфита (если большой net_dropout), или ёмкость
        sample_features = 1.#какая доля фичей приходит в субмодель. По дефолту ставится 0.4-0.6, но это если субмоделей как минимум 15. Сейчас их 1.
        path2model = "ern_model_E_composed.pth"
        max_tokens_in_loss = 4000#сколько пар эмбеддинг-токен проходит через tail adapter за один forward
    elif architecture == 'G':
        #сколько субмоделей выбрасывается из forward на обучении. Чем больше, тем лучше защита от оверфита. Если одна-две субмодели, net_dropout лучше занулять
        net_dropout_rate = 0.0#если десятки субмоделей и хотим огромную защиту от оверфита, то это число надо проставлять в 0.9-0.95
        #дропаут внутри субмоделей
        individ_dropout_rate = 0.025
        conservativity = 0.07#насколько сильно подавляем отклонение от старой стратегии
        accum_batch = 5000#1400#12000#60#для дачала сделайте 2
        #lr = 3e-7
        lr = 1e-4
        
        layer_configs = [2048, 2048, 2048, 2048, 2048, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 512, 512, 512, 512, 512, 512]
        composition_size = 1#число субмоделей класса ResNet. Чем больше их, тем больше или защита от оверфита (если большой net_dropout), или ёмкость
        sample_features = 1.#какая доля фичей приходит в субмодель. По дефолту ставится 0.4-0.6, но это если субмоделей как минимум 15. Сейчас их 1.
        path2model = "ern_model_G_composed.pth"
        max_tokens_in_loss = 4000#сколько пар эмбеддинг-токен проходит через tail adapter за один forward
    elif architecture == 'H':
        #сколько субмоделей выбрасывается из forward на обучении. Чем больше, тем лучше защита от оверфита. Если одна-две субмодели, net_dropout лучше занулять
        net_dropout_rate = 0.0#если десятки субмоделей и хотим огромную защиту от оверфита, то это число надо проставлять в 0.9-0.95
        #дропаут внутри субмоделей
        individ_dropout_rate = 0.025
        conservativity = 0.11#насколько сильно подавляем отклонение от старой стратегии
        accum_batch = 1000#1400#12000#60#для дачала сделайте 2
        #lr = 3e-7
        lr = 1e-6
        
        layer_configs = [2048] * 5 + [1024] * 7
        composition_size = 1#число субмоделей класса ResNet. Чем больше их, тем больше или защита от оверфита (если большой net_dropout), или ёмкость
        sample_features = 1.#какая доля фичей приходит в субмодель. По дефолту ставится 0.4-0.6, но это если субмоделей как минимум 15. Сейчас их 1.
        path2model = "ern_model_H_composed.pth"
        max_tokens_in_loss = 2300#сколько пар эмбеддинг-токен проходит через tail adapter за один forward
        use_memnets = True
        memnet_params={'num_heads':12, 'query_size':64, 'num_key_values':320, 'value_size':256}
    elif architecture == 'J':
        #сколько субмоделей выбрасывается из forward на обучении. Чем больше, тем лучше защита от оверфита. Если одна-две субмодели, net_dropout лучше занулять
        net_dropout_rate = 0.0#если десятки субмоделей и хотим огромную защиту от оверфита, то это число надо проставлять в 0.9-0.95
        #дропаут внутри субмоделей
        individ_dropout_rate = 0.025
        conservativity = 0.12#насколько сильно подавляем отклонение от старой стратегии
        accum_batch = 800#1400#12000#60#для дачала сделайте 2
        #lr = 3e-7
        lr = 6e-7
        
        layer_configs = [1024 * 2] * 2 + [1024] * 8
        composition_size = 2#число субмоделей класса ResNet. Чем больше их, тем больше или защита от оверфита (если большой net_dropout), или ёмкость
        sample_features = 1.#какая доля фичей приходит в субмодель. По дефолту ставится 0.4-0.6, но это если субмоделей как минимум 15. Сейчас их 1.
        path2model = "ern_model_J_composed.pth"
        max_tokens_in_loss = 2300#1200#сколько пар эмбеддинг-токен проходит через tail adapter за один forward
        use_memnets = True
        memnet_params={'num_heads':8, 'query_size':64, 'num_key_values':300, 'value_size':256}

    elif architecture == 'K':
        #сколько субмоделей выбрасывается из forward на обучении. Чем больше, тем лучше защита от оверфита. Если одна-две субмодели, net_dropout лучше занулять
        net_dropout_rate = 0.0#если десятки субмоделей и хотим огромную защиту от оверфита, то это число надо проставлять в 0.9-0.95
        #дропаут внутри субмоделей
        individ_dropout_rate = 0.025
        conservativity = 0.12#насколько сильно подавляем отклонение от старой стратегии
        accum_batch = 800#1400#12000#60#для дачала сделайте 2
        #lr = 3e-7
        lr = 6e-7
        
        layer_configs = [1024 * 2] * 2 + [1024] * 8
        composition_size = 2#число субмоделей класса ResNet. Чем больше их, тем больше или защита от оверфита (если большой net_dropout), или ёмкость
        sample_features = 1.#какая доля фичей приходит в субмодель. По дефолту ставится 0.4-0.6, но это если субмоделей как минимум 15. Сейчас их 1.
        path2model = "ern_model_K_composed.pth"
        max_tokens_in_loss = 4000#1200#сколько пар эмбеддинг-токен проходит через tail adapter за один forward
        use_memnets = True
        memnet_params={'num_heads':8, 'query_size':64, 'num_key_values':300, 'value_size':256}

    elif architecture == 'L':
        #сколько субмоделей выбрасывается из forward на обучении. Чем больше, тем лучше защита от оверфита. Если одна-две субмодели, net_dropout лучше занулять
        net_dropout_rate = 0.0#если десятки субмоделей и хотим огромную защиту от оверфита, то это число надо проставлять в 0.9-0.95
        #дропаут внутри субмоделей
        individ_dropout_rate = 0.025
        conservativity = 0.08#насколько сильно подавляем отклонение от старой стратегии
        accum_batch = 700#1900#0#10000#1400#12000#60#для дачала сделайте 2
        #lr = 3e-7
        lr = 2e-6
        #lr = 2e-5
        
        layer_configs = [1024 * 2] * 2 + [1024] * 8
        composition_size = 1#число субмоделей класса ResNet. Чем больше их, тем больше или защита от оверфита (если большой net_dropout), или ёмкость
        sample_features = 1.#какая доля фичей приходит в субмодель. По дефолту ставится 0.4-0.6, но это если субмоделей как минимум 15. Сейчас их 1.
        path2model = "ern_model_L_composed.pth"
        max_tokens_in_loss = 1200#1000#2000#1200#сколько пар эмбеддинг-токен проходит через tail adapter за один forward
        use_memnets = False
        memnet_params={'num_heads':8, 'query_size':64, 'num_key_values':300, 'value_size':256}

    elif architecture == 'M':
        #сколько субмоделей выбрасывается из forward на обучении. Чем больше, тем лучше защита от оверфита. Если одна-две субмодели, net_dropout лучше занулять
        #сколько субмоделей выбрасывается из forward на обучении. Чем больше, тем лучше защита от оверфита. Если одна-две субмодели, net_dropout лучше занулять
        net_dropout_rate = 0.0#если десятки субмоделей и хотим огромную защиту от оверфита, то это число надо проставлять в 0.9-0.95
        #дропаут внутри субмоделей
        individ_dropout_rate = 0.03
        conservativity = 0.15#насколько сильно подавляем отклонение от старой стратегии
        accum_batch = 400#600#800#400#270#230#450#1900#0#10000#1400#12000#60#для дачала сделайте 2
        #lr = 3e-7
        lr = 1e-6
        
        #layer_configs = [5000] + [4000] + [3200] + [2200] + [512] * 6
        #layer_configs = [5000] + [4000] + [512] * 12
        layer_configs = [320] * 1 + [160] * 1 + [64] * 6
        composition_size = 27#число субмоделей класса ResNet. Чем больше их, тем больше или защита от оверфита (если большой net_dropout), или ёмкость
        sample_features = 0.8#какая доля фичей приходит в субмодель. По дефолту ставится 0.4-0.6, но это если субмоделей как минимум 15. Сейчас их 1.
        path2model = "ern_model_M_composed.pth"
        max_tokens_in_loss = 40#170#250#2000#1200#сколько пар эмбеддинг-токен проходит через tail adapter за один forward
        use_memnets = False
        memnet_params={'num_heads':6, 'query_size':32, 'num_key_values':250, 'value_size':256}
        start_train = False
        
        use_moe = True
        router_layer_configs=[1024, 512, 256, 256, 256]
        top_k = 6
        inference_top_k = 6

    elif architecture == 'N':
        #сколько субмоделей выбрасывается из forward на обучении. Чем больше, тем лучше защита от оверфита. Если одна-две субмодели, net_dropout лучше занулять
        #сколько субмоделей выбрасывается из forward на обучении. Чем больше, тем лучше защита от оверфита. Если одна-две субмодели, net_dropout лучше занулять
        net_dropout_rate = 0.0#если десятки субмоделей и хотим огромную защиту от оверфита, то это число надо проставлять в 0.9-0.95
        #дропаут внутри субмоделей
        individ_dropout_rate = 0.05
        conservativity = 0.15#насколько сильно подавляем отклонение от старой стратегии
        accum_batch = 310#600#800#400#270#230#450#1900#0#10000#1400#12000#60#для дачала сделайте 2
        #lr = 3e-7
        lr = 3e-7
        
        #layer_configs = [5000] + [4000] + [3200] + [2200] + [512] * 6
        #layer_configs = [5000] + [4000] + [512] * 12
        layer_configs = [320] * 1 + [160] * 2 + [64] * 7
        composition_size = 25#число субмоделей класса ResNet. Чем больше их, тем больше или защита от оверфита (если большой net_dropout), или ёмкость
        sample_features = 0.8#какая доля фичей приходит в субмодель. По дефолту ставится 0.4-0.6, но это если субмоделей как минимум 15. Сейчас их 1.
        path2model = "ern_model_N_composed.pth"
        max_tokens_in_loss = 60#20#170#250#2000#1200#сколько пар эмбеддинг-токен проходит через tail adapter за один forward
        use_memnets = False
        memnet_params={'num_heads':6, 'query_size':32, 'num_key_values':250, 'value_size':256}
        start_train = False
        
        use_moe = True
        router_layer_configs=[1024, 512, 256, 256, 256]
        top_k = 7
        inference_top_k = 6

        

    #я подобрал перебором это значение batch_size и accum_batch, именно при нём быстрее всего проходим батч размера 2000. Я не знаю, почему, но это так.
    batch_size = 2

    
    if learnable_all:
        #опасный и тормозной режим
        batch_size = 2
        layer_configs = [512, 256]

In [11]:
#настройки датасета

if mode == 'pretrain':
    #претрейн
    #max_seq_len = 11000
    #max_seq_len4inference = 10950
    # max_seq_len = 6900
    # max_seq_len4inference = 6850
    max_seq_len = 6500
    max_seq_len4inference = 6450
    # max_seq_len = 3800
    # max_seq_len4inference = 3750
    # max_seq_len = 3600
    # max_seq_len4inference = 3550
else:
    #дообучение
    # max_seq_len = 1950
    # max_seq_len4inference = 1900
    # max_seq_len = 2050
    # max_seq_len4inference = 2000
    max_seq_len = 9050
    max_seq_len4inference = 9000

print('batch_size', batch_size)

batch_size 2


In [12]:
device

device(type='cuda', index=0)

In [13]:
tokenizer = AutoTokenizer.from_pretrained("unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit")#, cache_dir="D:\cache\huggingface\\"+ model_name)

if bits_per_number == 4:
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4"#, bnb_4bit_compute_dtype=torch.bfloat16
    )
elif bits_per_number == 8:
    bnb_config = BitsAndBytesConfig(
        load_in_8bit=True, bnb_8bit_use_double_quant=True, bnb_8bit_quant_type="nf8"#, bnb_8bit_compute_dtype=torch.bfloat16
    )
else:
    bnb_config = None

tokenizer.pad_token = tokenizer.eos_token
if learnable_all and not start_train:
    model = AutoModelForCausalLM.from_pretrained(
                  './last_modell',
                  #device_map="auto",
                  #device_map=device,
                  #torch_dtype=torch.bfloat16,
                  quantization_config=bnb_config,
                  cache_dir="D:\cache\huggingface\\"+ model_name)
else:
    model = AutoModelForCausalLM.from_pretrained(
                  model_name,
                  #device_map="auto",
                  #device_map=device,
                  #torch_dtype=torch.bfloat16,
                  quantization_config=bnb_config,
                  cache_dir="D:\cache\huggingface\\"+ model_name)
if use_gguf:
    model = PeftModelForCausalLM.from_pretrained(model, gguf_folder_name, load_in_4bit =True)
    if isinstance(model, PeftModelForCausalLM):
        model = model.merge_and_unload()


<>:22: SyntaxWarning: invalid escape sequence '\c'
<>:30: SyntaxWarning: invalid escape sequence '\c'
<>:22: SyntaxWarning: invalid escape sequence '\c'
<>:30: SyntaxWarning: invalid escape sequence '\c'
/tmp/ipykernel_2756843/2072397812.py:22: SyntaxWarning: invalid escape sequence '\c'
  cache_dir="D:\cache\huggingface\\"+ model_name)
/tmp/ipykernel_2756843/2072397812.py:30: SyntaxWarning: invalid escape sequence '\c'
  cache_dir="D:\cache\huggingface\\"+ model_name)
Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
/home/ssdovgan/main/lib/python3.12/site-packages/transformers/quantizers/auto.py:186: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)
`low_cpu_mem_usage` was

In [14]:
#model.lm_head
torch.save(model.lm_head.weight, "lin_model.pth")

In [15]:
# Создание Dataset
if mode == 'pretrain':
    dataset = InstructDatasetR("../data_tea/dataset_llm_full_flat.pkl", tokenizer, max_seq_len, cut=None)
else:
    dataset = InstructDatasetR("../data_tea/dataset_llm_full_instruct.pkl", tokenizer, max_seq_len, cut=None)
len(dataset)

1538455

In [16]:
if start_train:
    #создать модель
    torch.manual_seed(1)
    np.random.seed(1)
    random.seed(1)
    if use_moe:
        head = ensembles.MOE(input_size=embedding_size, 
                   out_size=cardinality, 
                   dropout_rate=individ_dropout_rate,
                   layer_configs=layer_configs, 
                   use_sigmoid_end=False, 
                   use_batchnorm=True, 
                   use_activation=True, 
                   activation=nn.LeakyReLU(), 
                   sample_features=sample_features, 
                   initial_num_experts=composition_size,
                   lin_bottleneck_size=None,
                   lin_model_add=nn.Linear(embedding_size, cardinality).to(device),
                   memnet_params=memnet_params,
                   use_memnets=use_memnets,
                   router_layer_configs=router_layer_configs,
                   top_k=top_k,
                   inference_top_k=inference_top_k,
                   unlock_last_model=True)
        head.inference_top_k = inference_top_k
        head.top_k = top_k
    else:
        head = ensembles.EResNetPro(input_size=embedding_size, 
                   out_size=cardinality, 
                   net_dropout_rate=net_dropout_rate, 
                   individ_dropout_rate=individ_dropout_rate,
                   layer_configs=layer_configs, 
                   use_sigmoid_end=False, 
                   use_batchnorm=True, 
                   use_activation=True, 
                   activation=nn.SiLU(),#nn.LeakyReLU(), 
                   sample_features=sample_features, 
                   composition_size=composition_size, 
                   lin_bottleneck_size=None,
                   lin_model_add=nn.Linear(embedding_size, cardinality).to(device),
                   memnet_params=memnet_params,
                   use_memnets=use_memnets,
                   exponential_layer_size=False)
    head.submodels[-1].weight = torch.nn.Parameter(torch.load( "lin_model.pth").to(device).to(torch.float32))
    head.submodels[-1].weight.requires_grad = learnable_linear_model
else:
    while 1:
        try:
            head = torch.load(path2model, weights_only=False, map_location=device)
            break
        except Exception:
            import time
            print(path2model, 'not loaded')
            time.sleep(20)

#собрать
if learnable_all:
    model.train()
    for param in model.parameters():
        try:
            param.requires_grad = True
            print('scss')
        except Exception as e:
            print(e)
else:
    model.eval()
head.train()
head.to(device)
head.by_submodels = True
if opt_type == 'adam':
    optimizer = torch.optim.Adam(head.parameters(), lr=lr)
else:
    momentum = 0.
    optimizer = torch.optim.SGD(head.parameters(), lr=lr*0.06, momentum=momentum)

In [17]:
def soft_clip(tens, mn, mx):
    mn = torch.tensor(mn).to(tens.device)
    mx = torch.tensor(mx).to(tens.device)
    tens[tens<mn] = torch.nn.Sigmoid()(tens[tens<mn]) + mn
    tens[tens>mx] = torch.nn.Sigmoid()(tens[tens>mx]) + mx
    return tens

In [18]:
loss_fct = nn.CrossEntropyLoss(reduction = 'none')

In [19]:
# Функция для подсчёта количества последних токенов, равных padding_token
def count_padding(tensor, padding_token):
    counts = []
    for row in tensor:
        count = 0
        for token in reversed(row):
            if token.item() == padding_token:
                count += 1
            else:
                break
        counts.append(count)
    return min(counts)

In [20]:
import sequential_models
sequential_models.count_trainable_parameters(head)

(3515754341, 525336576)

In [21]:
#заморозить первые N слоёв выходного резнета
depth2freeze = 0#64
print('resnet depth', len(head.submodels[0].layers), 'depth2freeze', depth2freeze)
for i in range(len(head.submodels) - 1):
    for l_num in range(len(head.submodels[0].layers)):
        for p in head.submodels[i].layers[l_num].parameters():
            p.requires_grad = l_num >= depth2freeze
            #а какие не надо - не замораживать!

# for l_num in range(depth2freeze + 2):
#     for p in head.submodels[0].layers[l_num].parameters():
#         print(p.requires_grad)

resnet depth 44 depth2freeze 0


In [22]:
proba_drop_idx = 0.04
noise_coef = 0.04#0.03

In [ ]:
#проинференсить модель на батчах, сделать датасет для постпроцессинга
print('batch_size', batch_size)
i_pointer = 0

batch_accum_counter = 0
loss_array = []
metr_array = []
while 1: 
    input_ids_full = []
    labels_full = []
    weights_full = []
    #for i in range(batch_size):
    i = 0
    offset_counter = 0
    while i < batch_size:
        offset_counter += 1
        sample_cur  = dataset[i_pointer + i]
        input_ids = sample_cur['input_ids']
        labels = sample_cur['labels']
        weights = sample_cur['mult']

        if labels.numel() < batch_size * 10:
            #штраф на короткие строки для сокращения шума
            weights *= 0.7
        if labels.numel() < batch_size * 20:
            #штраф на короткие строки для сокращения шума
            weights *= 0.8
        if labels.numel() < batch_size * 50:
            #штраф на короткие строки для сокращения шума
            weights *= 0.8
        
        input_ids_len = len(input_ids)
        labels_len = len(labels)
        input_ids = input_ids[input_ids!=padding_token]
        
        drop_idx = torch.rand_like(input_ids.to(torch.float)) < proba_drop_idx
        input_ids = input_ids[~drop_idx]
        
        labels = labels[labels!=padding_token]
        padding_size = input_ids_len + labels_len - len(input_ids) - len(labels)
        input_ids_cur = torch.cat([input_ids, labels, torch.zeros(padding_size, dtype=torch.int32) + padding_token])[:max_seq_len4inference]
        labels_cur = torch.cat([input_ids * 0 + padding_token, labels, torch.zeros(padding_size, dtype=torch.int32) + padding_token])[:max_seq_len4inference]
        #print('fact_size', np.min([len(input_ids) + len(labels), max_seq_len4inference]), len(input_ids) + len(labels))
        if torch.all(labels_cur) == padding_token:
            print('empty label')
            continue
        i += 1
        
        input_ids_full.append(input_ids_cur[:-1])
        labels_full.append(labels_cur[1:])
        weights_full.append(weights)

        # if not torch.any(input_ids!=0):
        #     #X + Y
        #     print(list(input_ids_cur.numpy()))
        #     print(list(labels_cur.numpy()))
        #     1/0
    i_pointer += i #should be batch_size
    #print('A', pd.Timestamp.now())
    with torch.no_grad():
        input_ids_full = torch.stack(input_ids_full).to(device)
        labels_full = torch.stack(labels_full).to(device)
        weights_full = torch.stack(weights_full).to(device).T

        if 1:
            cnt_pads = count_padding(labels_full, padding_token)
            if cnt_pads >= input_ids_full.shape[1] - 5:
                continue
            if cnt_pads > 0:
                input_ids_full = input_ids_full[:, :-cnt_pads]
                labels_full = labels_full[:, :-cnt_pads]
            
        
        
        
        #
        #outp = model.forward(input_ids_full, attention_mask=None, labels=labels_full, token_type_ids=None, write_caches=False, read_caches=False, return_states=True)
        # head.by_submodels = False
        # head.training = False
        # model.lm_head = head
        # model.lm_head.half()
        # outp = model.forward(input_ids_full, output_hidden_states=True, return_dict=False)

        outp = model.forward(input_ids_full, output_hidden_states=True, return_dict=True)
        outp['states'] = outp['hidden_states'][-1]
    # {"loss": loss_agg, "logits": logits, "states":embeddings}
        outp['states'] = outp['states'][:, :input_ids_full.shape[1]]
        state_cur = outp['states'].reshape([outp['states'].shape[0] * outp['states'].shape[1], outp['states'].shape[2]])#.cpu().to(torch.float16).numpy()
        state_cur += noise_coef * torch.std(state_cur) * torch.randn_like(state_cur)
        weights_full2d = torch.vstack([weights_full] * labels_full.shape[-1]).to(device).T
        labels_cur = labels_full.ravel()#.cpu().numpy()
        weights_cur = weights_full2d.ravel()

        del labels_full
        del weights_full2d
        del weights_full
        del outp
        idx = torch.isin(labels_cur, torch.tensor(forbidden_tokens_list).to(device))
        labels_cur = labels_cur[~idx]
        state_cur = state_cur[~idx]
        weights_cur = weights_cur[~idx]
        # if len(labels_cur) > max_tokens_in_loss:
        #     indices = np.random.choice(len(labels_cur), size=max_tokens_in_loss, replace=False)
        #     labels_cur = labels_cur[indices]
        #     state_cur = state_cur[indices]
    #print('B', pd.Timestamp.now())
    if state_cur.shape[0] == 0:
        continue
    for x_start_pointer in range(0, state_cur.shape[0], max_tokens_in_loss): 
        logits, lst_logits = head(state_cur.to(torch.float32)[x_start_pointer: x_start_pointer + max_tokens_in_loss])
        
        ce_loss = loss_fct(logits, labels_cur[x_start_pointer: x_start_pointer + max_tokens_in_loss].view(-1))
        ce_loss = ce_loss * weights_cur[x_start_pointer: x_start_pointer + max_tokens_in_loss].to(device)
        ce_loss = soft_clip(ce_loss, -0.9, 10.)
        loss_ampl_head = torch.nanmean(torch.abs(lst_logits[-1] - logits)) / (torch.std(lst_logits[-1]) + 1)
        
        loss = torch.nanmean(ce_loss) * (1./(conservativity + 1.)) + loss_ampl_head * (conservativity/(conservativity + 1.))
        loss.backward()
        l = len(lst_logits)
        for j in range(l - 1, -1, -1):
            del lst_logits[j]
        #print('-C', pd.Timestamp.now())
        #del lst_logits
    del state_cur
    #print('D', pd.Timestamp.now())

    acc = torch.mean((torch.argmax(logits, axis=-1) == labels_cur[x_start_pointer: x_start_pointer + max_tokens_in_loss]).to(torch.float16))
    del logits
    del labels_cur
    batch_accum_counter += 1
    loss_array += [torch.mean(ce_loss).item()]
    metr_array += [acc.item()]
    if batch_accum_counter >= accum_batch:
        optimizer.step()
        optimizer.zero_grad()
        torch.cuda.empty_cache()
        print('ce_loss', np.round(np.mean(loss_array), 3), 'loss_ampl_head', np.round(loss_ampl_head.item(), 4), i_pointer, 'from', len(dataset), np.round(100 * i_pointer/len(dataset), 5), '%', 'acc', np.mean(metr_array), pd.Timestamp.now())
        loss_array = []
        metr_array = []
        batch_accum_counter = 0

        
        
        if (np.random.rand() < 1 and accum_batch > 100) or (np.random.rand() < 0.1 and accum_batch <= 100):
            head.training = False
            torch.save(head, path2model)
            if np.random.rand()<0.2:
                torch.save(head, path2model + '.back')
            head.training = True
            if learnable_all:
                model.eval()
                model.save_pretrained("./last_modell")
                model.train()
                for param in model.parameters():
                    try:
                        param.requires_grad = True
                    except Exception as e:
                        pass 
            

batch_size 2


/tmp/ipykernel_2756843/2814761306.py:63: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3683.)
  weights_full = torch.stack(weights_full).to(device).T


ce_loss 1.539 loss_ampl_head 0.0426 620 from 1538455 0.0403 % acc 0.6553671559979839 2025-07-15 00:50:34.699037


In [ ]:
#4.5 секунд на строку в среднем, длина 4200, adam
#2 сек на строку длины 3800 с adam
#0.8 сек на строку длины 3800, sgd без momentum (?!)
# сек на строку длины 4200, sgd без momentum
#2 сек на строку длины 3600 с adam

In [ ]:
1

In [ ]:
#если принудительно стопнули, можно добить батч
optimizer.step()
optimizer.zero_grad()
torch.cuda.empty_cache()
print('ce_loss', np.round(np.mean(loss_array), 3), 'loss_ampl_head', np.round(loss_ampl_head.item(), 4), i_pointer, 'from', len(dataset), np.round(100 * i_pointer/len(dataset), 5), '%', 'acc', np.mean(metr_array), pd.Timestamp.now())

In [ ]:
#сохранение tail adapter-а вручную
head.training = False
head.by_submodels = False
torch.save(head, path2model)

if learnable_all:
    model.eval()
    model.save_pretrained("./last_modell")

In [ ]:
head